In [2]:
!pip install --upgrade --quiet openai pyautogen

In [3]:
import autogen

config_list =[
    {   
        "model" : "gemma:2b", 
        "api_key": "EMPTY", 
        "max_tokens":1000,
        "base_url":f"http://localhost:11434/v1"},
    {
        "model": "gpt-4",
        "api_key": "<your Azure OpenAI API key here>",
        "base_url": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2024-02-15-preview"
    },
    {
        "model": "gpt-4-32k",
        "api_key": "<your Azure OpenAI API key here>",
        "base_url": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2024-02-15-preview"
    }
]

gemmaConfig = config_list[0]
gemmaConfig

{'model': 'gemma:2b',
 'api_key': 'EMPTY',
 'max_tokens': 1000,
 'base_url': 'http://localhost:11434/v1'}

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config=False,
    human_input_mode="TERMINATE",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)
procedure = autogen.AssistantAgent(
    name="Procedure",
    system_message="""You an advanced AI acting as the an mathematics procedural assistant;
    you will take a given math problem written in LaTex format, create a high fidelity summary of what the problem is asking you to achieve;
    then break the task down into a list of individual steps that must be completed to solve the task.
                    """,
    llm_config=llm_config,
    description="""This is a math problem procedure assistant who is capable of taking a problem in LaTex format; summarizing the question; and creating a procedure for solving it.
                   The procedure assistant is open to any comments and recommendations for improving the summary and procedure.
                   Ask the procedure assistant to iterate every time when there is a new change recommendation from editor.
                """
)
editor = autogen.AssistantAgent(
    name="Editor",
    system_message="""You review the math questions and procedure proposals and give change recommendations to make the procedure and summary more aligned with the original question being asked,
    for example, revising to make sure the correct solution format is being protrayed.
                      You never write or revise procedures by yourself.
                       """,
    llm_config=llm_config,
    description="""This is an editor who reviews the procedure drafts of mathematics procedural assistants and provides change ideas.
                    The editor should be called every time the mathematics procedural assistant provides a version of a math procedure post.
                """
)
solver = autogen.AssistantAgent(
    name="Solver",
    system_message=""" You are an advanced mathematician;
    You will take a given math problem summary and a procedure for solving it;
    and you will follow the procedure to solve the math problem correctly.
                       """,
    llm_config=llm_config,
    description="""This is an elite math problem solver who reviews math procedures of procedure writing assistants and provides the solution to the problem.
                    The solver should be called every time the procedure assistant provides a version of procedure summary and steps.
                """
)


In [ ]:
groupchat = autogen.GroupChat(agents=[user_proxy, procedure, editor, solver], messages=[], max_round=6)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)